# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head(15)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,papatowai,-46.5619,169.4708,8.86,90,47,2.59,NZ,1721513369
1,1,san antonio de pale,-1.4014,5.6325,22.79,74,53,4.61,GQ,1721513370
2,2,meadow lakes,61.6247,-149.6011,24.05,59,0,1.54,US,1721513372
3,3,ust-nera,64.5667,143.2000,13.10,97,100,2.87,RU,1721513373
4,4,wailua homesteads,22.0669,-159.3780,27.85,67,40,5.14,US,1721513374
5,5,mount pearl,47.5166,-52.7813,18.37,76,2,1.47,CA,1721513375
6,6,rio san juan,19.6439,-70.0763,27.86,83,99,4.33,DO,1721513376
7,7,tarawa,1.4190,172.9840,30.05,74,40,6.17,KI,1721513377
8,8,cabo san lucas,22.8909,-109.9124,30.66,62,0,5.94,MX,1721512863
9,9,santa maria da boa vista,-8.8078,-39.8256,26.18,41,85,6.93,BR,1721513264


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
# Import necessary libraries
import pandas as pd
import geoviews as gv
import geoviews.tile_sources as gvts

# Assuming you have the city_data_df DataFrame loaded with the required data

# Create a GeoViews Points plot with humidity as the size of the points
map_plot = city_data_df.hvplot.points(
    'Lng', 'Lat', size='Humidity',
    geo=True, tiles=gvts.CartoLight, frame_width=800, frame_height=600
)

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Define the condition for the ideal temperature range
ideal_conditions = (city_data_df['Max Temp'] > 15) & (city_data_df['Max Temp'] < 20)

# Filter the DataFrame based on the ideal temperature conditions using the boolean mask from ideal_conditions
filtered_cities_df = city_data_df[ideal_conditions]

# Display the filtered DataFrame
print(filtered_cities_df)

     City_ID                City      Lat       Lng  Max Temp  Humidity  \
5          5         mount pearl  47.5166  -52.7813     18.37        76   
12        12            fisterra  42.9049   -9.2629     16.68        78   
13        13             udachny  66.4167  112.4000     17.48        76   
32        32         radviliskis  55.8167   23.5333     16.03        94   
34        34              nemuro  43.3236  145.5750     16.75        99   
..       ...                 ...      ...       ...       ...       ...   
558      558  severo-yeniseyskiy  60.3725   93.0408     15.09        99   
563      563               inari  68.9060   27.0288     17.11        63   
568      568           kalininsk  51.5000   44.4758     17.60        71   
569      569              luanda  -8.8368   13.2343     19.64        90   
573      573            cananeia -25.0147  -47.9267     15.63        89   

     Cloudiness  Wind Speed Country        Date  
5             2        1.47      CA  1721513375  

### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Create a new DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column "Hotel Name" to the DataFrame to store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ""

# Display sample data
print(hotel_df.head())

                  City Country      Lat       Lng  Humidity Hotel Name
0            papatowai      NZ -46.5619  169.4708        90           
1  san antonio de pale      GQ  -1.4014    5.6325        74           
2         meadow lakes      US  61.6247 -149.6011        59           
3             ust-nera      RU  64.5667  143.2000        97           
4    wailua homesteads      US  22.0669 -159.3780        67           


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel

        # Set parameters for the API request
params = {
    "categories":"accommodation.hotel",
    "radius": 10000,  # 10,000 meters radius
    "apiKey": geoapify_key,
    "limit": 20,  # Limit to 20 results
    }


# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():

    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lat},{lng},100000"  # Assuming 10,000 meters radius
    params["bias"] = f"proximity:{lat},{lng}"
    params["limit"]=20
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
papatowai - nearest hotel: No hotel found
san antonio de pale - nearest hotel: Even Me Hotel
meadow lakes - nearest hotel: No hotel found
ust-nera - nearest hotel: No hotel found
wailua homesteads - nearest hotel: No hotel found
mount pearl - nearest hotel: No hotel found
rio san juan - nearest hotel: No hotel found


KeyboardInterrupt: 

### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [8]:
pip install folium

Note: you may need to restart the kernel to use updated packages.


In [9]:
import folium

# Create a map centered at a specific location
m = folium.Map(location=[40.7128, -74.0060], zoom_start=4)

# Add markers for cities with hotel information
for index, row in hotel_df.iterrows():
    popup_content = f"City: {row['City']}<br>Hotel: {row['Hotel Name']}<br>Country: {row['Country']}"
    folium.Marker(location=[row['Lat'], row['Lng']], popup=popup_content).add_to(m)

m